In [36]:
import pandas as pd
from pathlib import Path
import numpy as np
import xarray as xr
from tqdm.auto import tqdm

In [2]:
from icesea2.experiments import gen_lhs_from_dict, norm_to_values, topography_to_icesheet_nc, gen_margin_input_files, gen_run_configs
from icesea2.shearstress import make_shear_stress_map
from icesea2.margins import shp_series_to_icesheet
from icesea2.models import IcesheetModel
from icesea2.gridtools import grid_icesheet_pool, grid_icesheet_series
 

In [4]:
shear_stress_map_name = "ely_victor_eurasia"
shear_stress_map_file = "/nfs/annie/cm15ogp/phd/data/shear_stress/eurasia/regions.shp"

In [5]:
dx = 5000
dy = 5000
x_min = -1265453.0
x_max = 4159547.0
y_min = -4722734.8
y_max = 1352265.2
grid_x = np.arange(x_min, x_max+dx, dx)
grid_y = np.arange(y_min, y_max+dy, dy)

grid_crs = '+ellps=WGS84 +proj=laea +lon_0=0.0 +lat_0=90 +x_0=0.0 +y_0=0.0 +no_defs'

In [6]:
times = [-22000,-20000,-18000,-16000]

In [7]:
shear_stress_parameters_var_ranges = {
    "g_lgm_ice_streams_ice_stream": [5000,20000],
    "g_sediment_marine_sediment": [10000,30000],
    "g_sediment_onshore_sediment": [30000,100000],
    "g_sediment_bedrock": [100000,150000],
    "p_ice_stream_interior_dist": [0,1000],
    "p_cold_ice_shear_stress": [120000,200000],
    "p_cold_ice_interior_dist": [1000,300]
}
shear_stress_parameters_const = {
    "m_base_map_name": shear_stress_map_name,
    "m_base_map_file": shear_stress_map_file,
    "m_gauss_blur": 1,
    "m_base_value": 5000,
    "m_combine_pattern": "(g_sediment | (g_lgm_ice_streams * p_ice_stream)) | p_cold_ice",
    "g_layers": ["sediment", "lgm_ice_streams"],
    "p_processes": ["ice_stream", "cold_ice"],
}

In [10]:
! ls ../../glac1d

glac1d_icesheet.nc  lhs_norm.csv    outputs_gridded
inputs		    lhs_values.csv  outputs_raw


In [11]:
pd.read_csv("../../glac1d/lhs_values.csv")

,g_lgm_ice_streams_ice_stream,g_sediment_marine_sediment,g_sediment_onshore_sediment,g_sediment_bedrock,p_ice_stream_interior_dist,p_cold_ice_shear_stress,p_cold_ice_interior_dist
0,10341.296115,20023.430323,65682.239616,147281.903649,550.344096,181545.035528,558.201426
1,5647.082453,22362.578744,68146.823648,118698.419396,248.464200,187587.423718,592.750215
2,16223.214521,25587.443663,64063.151829,105938.286079,569.421152,133108.145756,409.445132
3,16268.621002,10621.660822,89495.616465,112868.203696,956.861359,139837.614592,985.961424
4,16343.997410,28497.833312,61396.313072,136961.430637,657.071250,131477.750325,733.585092
...,...,...,...,...,...,...,...
195,17986.018771,16566.140779,89551.298339,135051.869153,585.212799,188163.825018,406.732650
196,12985.963242,13817.539886,40187.372838,114919.098671,974.009705,193241.011662,797.565523
197,19833.418076,14276.209623,86261.304957,123612.279709,305.557201,167026.979004,841.924258
198,19631.946679,25869.234886,78779.229855,137347.481277,858.346997,130524.578350,308.033664


In [26]:
min_vals

array([  5000,  10000,  30000, 100000,      0, 120000,   1000])

In [ ]:
norm = (vals - min)/(max - min)

In [31]:
exp_vars = pd.DataFrame(corner_values, columns=list(shear_stress_parameters_var_ranges.keys()))
exp_vars

,g_lgm_ice_streams_ice_stream,g_sediment_marine_sediment,g_sediment_onshore_sediment,g_sediment_bedrock,p_ice_stream_interior_dist,p_cold_ice_shear_stress,p_cold_ice_interior_dist
0,5000,10000,30000,100000,0,120000,1000
1,5000,10000,30000,100000,0,120000,300
2,5000,10000,30000,100000,0,200000,1000
3,5000,10000,30000,100000,0,200000,300
4,5000,10000,30000,100000,1000,120000,1000
...,...,...,...,...,...,...,...
123,20000,30000,100000,150000,0,200000,300
124,20000,30000,100000,150000,1000,120000,1000
125,20000,30000,100000,150000,1000,120000,300
126,20000,30000,100000,150000,1000,200000,1000


In [33]:
! pwd

/nfs/annie/cm15ogp/phd/paper1/paper1_data/icesheet_runs/ice6g/corners


In [34]:
margins_name = "smooth_v2_topoclip_mod_-1km"
margins_dir = "/nfs/annie/cm15ogp/phd/data/ice_margins/ice6g/smooth_v2_topoclip_mod_-1km"
run_dir = "/nfs/annie/cm15ogp/phd/paper1/paper1_data/icesheet_runs/ice6g/corners/"

t_var_topo = True
e_var_topo = False
e_var_ss = True
t_var_ss = True
e_var_margins = False

In [37]:
for time in tqdm(times):
    margin_filename = str(Path(margins_dir) / str(time) / "margin.shp")
    shear_stress_parameters_margins = {
        "p_margin_name": margins_name,
        "p_margin_time": time,
        "p_margin_file": margin_filename,
    }
    for exp_name, values in tqdm(exp_vars.iterrows(), total=len(exp_vars)):
        inputs_dir_shear_stress = (
            Path(run_dir) / 
            'inputs' / 
            'shearstress' / 
            str(time) / 
            str(exp_name)
        )
        inputs_dir_shear_stress.mkdir(exist_ok=True, parents=True)
        shear_stress_parameters_var = values.to_dict()
        shear_stress_parameters = {
            **shear_stress_parameters_const, 
            **shear_stress_parameters_var, 
            **shear_stress_parameters_margins,
        }

        make_shear_stress_map(
            grid_x = grid_x, 
            grid_y = grid_y, 
            grid_crs = grid_crs, 
            parameters = shear_stress_parameters,
            save=str(inputs_dir_shear_stress / "shearstress.nc")
        )


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

In [48]:
# ice sheet margins
# inputs_dir_margins = (run_dir / 'inputs/margins')

inputs_dir_margins = (
    Path(run_dir) / 
    'inputs' / 
    'margins'
)
inputs_dir_margins.mkdir(exist_ok=True, parents=True)

shp_series_to_icesheet(
    margins_dir, 
    grid_crs, 
    str(inputs_dir_margins), 
    times=times
)

#split_margins = gen_margin_input_files(inputs_dir_margins, times)

In [49]:
topography_name = "ice6g_laea_5km_eurasia"
topography_nc = "/nfs/annie/cm15ogp/phd/data/ice_models/ice6g/ice6g_laea_5km_eurasia.nc"


In [50]:
# topography
inputs_dir_topography = (Path(run_dir) / "inputs/topography")
inputs_dir_topography.mkdir(exist_ok=True, parents=True)
topography_to_icesheet_nc(topography_nc=topography_nc, output_dir=inputs_dir_topography, times=times)

In [51]:
t_var_topo = True
e_var_topo = False
e_var_ss = True
t_var_ss = True
t_var_margins = True
e_var_margins = False

In [52]:
run_configs = []
for member, config in exp_vars.iterrows():
    config = config.to_dict()
    run_config = gen_run_configs(
        run_dir = str(run_dir),
        times = times,
        member_name = str(member),
        t_var_topo=t_var_topo,
        e_var_topo=e_var_topo,
        t_var_ss=t_var_ss,
        e_var_ss=e_var_ss,
        e_var_margins=e_var_margins
    )
    run_configs = run_configs + run_config

In [53]:
ism = IcesheetModel("/nfs/annie/cm15ogp/code/icesheet/binaries/icesheet")

In [54]:
ism.run_pool(run_configs, cores=24)